Website Summarizer

In [39]:

import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('DEEPSEEK_API_KEY')
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
#elif not api_key.startswith("sk-proj-"):
#    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif not api_key.startswith("sk-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [43]:
from openai import OpenAI
import os
from dotenv import load_dotenv

def simple_chat(messages):
    
    load_dotenv(override=True)
    #api_key = os.getenv('OPENAI_API_KEY')
    api_key = os.getenv('DEEPSEEK_API_KEY')
    
    if not api_key:
        return "Error: DEEPSEEK_API_KEY environment variable is not set"
    

    client = OpenAI(
        api_key=api_key,
        base_url="https://api.deepseek.com/v1"
    )

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,  
            temperature=0.7,
            max_tokens=1000
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Website 
website = "www.muriras.com"

# prompts
system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""



In [44]:
#Getting website contents
import requests
from bs4 import BeautifulSoup

def get_website_content(url):
    try:
        
        if not url.startswith(('http://', 'https://')):
            url = 'https://' + url
            
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Removing script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
            
        # Getting text content
        text = soup.get_text()
        
        # Cleaning up the text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        return text
    except Exception as e:
        return f"Error fetching website content: {str(e)}"



In [45]:
# Main execution
if __name__ == "__main__":
    
    
    # Getting website content
    website_content = get_website_content(website)
    
   
    if website_content.startswith("Error"):
        print(f"Failed to fetch website: {website_content}")
    else:
        # Create the messages for the chat
        user_prompt = user_prompt_prefix + website_content
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
        # Get the summary
        summary = simple_chat(messages)
        print("Website Summary:")
        print(summary)

Website Summary:
### 🖥️ Muriras Ltd: Because Your Business Needs More Than Just a Fancy Logo

Welcome to Muriras Ltd, where they’ve apparently solved every business problem you didn’t know you had—from HR headaches to real estate woes, and even insurance collections (because who doesn’t dream of automating premium tracking?).

**What They Offer (in a Nutshell):**
- **HR Software:** For when you’re tired of remembering your employees’ names.
- **Real Estate Management:** So you can list properties while pretending you’re on *Selling Sunset*.
- **Payment Systems:** Because manually counting cash is so 1999.
- **Insurance Collections:** Automate nagging your clients for money—efficiency at its finest!
- **Point of Sale:** Real-time updates so you’ll know the instant you’re out of avocado toast.

**Their Big Vision:** To “empower” your business with “smart software,” though they forgot to mention it also comes with a side of buzzwords like “streamline,” “integrate,” and “transform.”

**New